<a href="https://colab.research.google.com/github/Janina712/RhythmMetrics_Duration/blob/main/4_Compute_Rhythm_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set-Up**

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pws_IDs_frog = ['24fa']
control_IDs_frog = []
pws_IDs_reading = ['24fa']
control_IDs_reading = []

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/

/content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics


In [ ]:
# PWS vs. Control
reading_vowels_fluent = pd.read_excel("matchedVowels_reading_FLUENT.xlsx")
reading_consonants_fluent = pd.read_excel("matchedConsonants_reading_FLUENT.xlsx")
frog_vowels_fluent = pd.read_excel("matchedVowels_frog_FLUENT.xlsx")
frog_consonants_fluent = pd.read_excel("matchedConsonants_frog_FLUENT.xlsx")

In [ ]:
#set up result path to save along the way
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics

dir = "4.MetricsResults"
if os.path.exists(dir) == False:
  os.mkdir(dir)

%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/4.MetricsResults/

/content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics
/content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/4.MetricsResults


# **Define Functions**

In [ ]:
def split_groups(df):
  pws = df[df["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
  pws.index = range(len(pws.index)) # reset index
  pws.drop(['Unnamed: 0'], axis=1 , inplace = True)

  control = df[df["Group"] == "Control"]  # get subset of sound dataframe that corresponds to current participant
  control.index = range(len(control.index)) # reset index
  control.drop(['Unnamed: 0'], axis=1 , inplace = True)
  return(pws, control)

In [ ]:
def split_fluency(df):

  for i in range(0,len(df)):
    df["FluencyStatus"][i] = df["FluencyStatus"][i].lower().strip()

  fluent = df[df["FluencyStatus"] == "fluent"]  # get subset of sound dataframe that corresponds to current participant
  fluent.index = range(len(fluent.index)) # reset index
  fluent.drop(['Unnamed: 0'], axis=1 , inplace = True)

  disfluent = df[df["FluencyStatus"] == "disfluent"]  # get subset of sound dataframe that corresponds to current participant
  disfluent.index = range(len(disfluent.index)) # reset index
  disfluent.drop(['Unnamed: 0'], axis=1 , inplace = True)
  return(fluent, disfluent)

In [ ]:
def computenPVI(df_vowels):
  participant = []                                                                                                                                                ## group  -
  for p in df_vowels["ID"].unique():                                                                                                                              ## participant  --
      subset_ID = df_vowels[df_vowels["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset index
      breath_group = []
      for BG in subset_ID["Breath.Group"].unique():                                                                                                               ## Breath Group  ---
        subset_BG = subset_ID[subset_ID["Breath.Group"] == BG]
        subset_BG.index = range(len(subset_BG.index))
        sum = 0
        for m in range(1, len(subset_BG)):                                                                                                                        ## vowel            ----
          subsum = (subset_BG["Duration"][m-1] - subset_BG["Duration"][m])/((subset_BG["Duration"][m-1] + subset_BG["Duration"][m])/2)
          sum = sum + abs(subsum)                                                                                                                                 ## vowel            ----
        if len(subset_BG) < 2:                                                                                                                                    ## Breath Group    ---
          continue
        else:
          BG_nPVI = 100 * (sum/(len(subset_BG)-1))
          breath_group.append(BG_nPVI)
      participant.append(breath_group)                                                                                                                            ## participant  --                                                                                                                                    ## group   -
  participant = pd.DataFrame(participant)
  group = []
  for k in range(0, len(participant)):
    participant_avg = participant.iloc[k].mean()
    group.append(participant_avg)
  group = pd.DataFrame(group, columns = ["nPVI"])
  return(group)

In [ ]:
def computerPVI(df_consonants):
  participant = []                                                                                                                                                ## group  -
  for p in df_consonants["ID"].unique():                                                                                                                              ## participant  --
      subset_ID = df_consonants[df_consonants["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset index
      breath_group = []
      for BG in subset_ID["Breath.Group"].unique():                                                                                                               ## Breath Group  ---
        subset_BG = subset_ID[subset_ID["Breath.Group"] == BG]
        subset_BG.index = range(len(subset_BG.index))
        sum = 0
        for m in range(1, len(subset_BG)):                                                                                                                        ## vowel            ----
          subsum = subset_BG["Duration"][m-1] - subset_BG["Duration"][m]
          sum = sum + abs(subsum)                                                                                                                                 ## vowel            ----
        if len(subset_BG) < 2:                                                                                                                                    ## Breath Group    ---
          continue
        else:
          BG_rPVI = sum/(len(subset_BG)-1)
          breath_group.append(BG_rPVI)
      participant.append(breath_group)                                                                                                                            ## participant  --                                                                                                                                    ## group   -
  participant = pd.DataFrame(participant)
  group = []
  for k in range(0, len(participant)):
    participant_avg = participant.iloc[k].mean()
    group.append(participant_avg)
  group = pd.DataFrame(group, columns = ["rPVI"])
  return(group)

In [ ]:
def computePercV(df_vowels, df_consonants):
  participant = []
  for p in df_vowels["ID"].unique():
      subset_ID = df_vowels[df_vowels["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset index
      numberBG = len(subset_ID["Breath.Group"].unique())
      subset_cons = df_consonants[df_consonants["ID"] == p]
      subset_cons.index = range(len(subset_cons.index))
      Vsums = np.zeros(numberBG)
      Csums = np.zeros(numberBG)
      count_v = 0
      for i in subset_ID["Breath.Group"].unique():
        count_v = count_v+1
        for j in range(0, len(subset_ID)):
          if subset_ID["Breath.Group"][j] == i:
            Vsums[count_v-1] = Vsums[count_v-1] + subset_ID["Duration"][j]
      count_c = 0
      for i in subset_ID["Breath.Group"].unique():
        count_c = count_c+1
        for j in range(0, len(subset_cons)):
          if subset_cons["Breath.Group"][j] == i:
            Csums[count_c-1] = Csums[count_c-1] + subset_cons["Duration"][j]
      VCsums = Vsums.mean() + Csums.mean()                                                         ## no mean??
      percV = Vsums.mean()/VCsums
      percV = percV[~np.isnan(percV)]
      percV_avg = percV.mean()
      participant.append([percV_avg])
  participant = pd.DataFrame (participant, columns = ['%V'])
  return(participant)

In [ ]:
def computeDeltaC(df_consonants):
  participant = []
  for p in df_consonants["ID"].unique():
    subset_ID = df_consonants[df_consonants["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
    subset_ID.index = range(len(subset_ID.index)) # reset index
    numberBG = len(subset_ID["Breath.Group"].unique())
    grouped_cons = subset_ID.groupby("Breath.Group")
    grouped_lists = grouped_cons["Duration"].apply(list)
    grouped_lists = grouped_lists.reset_index()
    deltaC = np.zeros(len(grouped_lists))
    for i in range(0,len(grouped_lists)):
      deltaC[i] = np.std(grouped_lists["Duration"][i])
    deltaC_avg = deltaC.mean()
    participant.append([deltaC_avg])
  participant = pd.DataFrame (participant, columns = ['DeltaC'])
  return(participant)

In [ ]:
def computeDeltaV(df_vowels):
    participant = []
    for p in df_vowels["ID"].unique():
      subset_ID = df_vowels[df_vowels["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset index
      numberBG = len(subset_ID["Breath.Group"].unique())
      grouped_vowels = subset_ID.groupby("Breath.Group")
      grouped_lists = grouped_vowels["Duration"].apply(list)
      grouped_lists = grouped_lists.reset_index()
      deltaV = np.zeros(len(grouped_lists))
      for i in range(0,len(grouped_lists)):
        deltaV[i] = np.std(grouped_lists["Duration"][i])
      deltaV_avg = deltaV.mean()
      participant.append([deltaV_avg])
    participant = pd.DataFrame (participant, columns = ['DeltaV'])
    return(participant)

In [ ]:
def computeVarcoC(df_consonants, deltaC):
    n = -1
    participant = []
    for p in df_consonants["ID"].unique():
      n = n +1
      subset_ID = df_consonants[df_consonants["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset
      numberBG = len(subset_ID["Breath.Group"].unique())
      grouped_consonants = subset_ID.groupby("Breath.Group")
      grouped_lists = grouped_consonants["Duration"].apply(list)
      grouped_lists = grouped_lists.reset_index()
      Cavg = np.zeros(len(grouped_lists))
      for i in range(0,len(grouped_lists)):
        Cavg[i] = np.mean(grouped_lists["Duration"][i])
      VarcoC = deltaC["DeltaC"][n]/Cavg
      VarcoC_avg = VarcoC.mean()
      participant.append([VarcoC_avg])
    participant = pd.DataFrame (participant, columns = ['VarcoC'])
    return(participant)

In [ ]:
def computeVarcoV(df_vowels, deltaV):
    n = -1
    participant = []
    for p in df_vowels["ID"].unique():
      n = n +1
      subset_ID = df_vowels[df_vowels["ID"] == p]  # get subset of sound dataframe that corresponds to current participant
      subset_ID.index = range(len(subset_ID.index)) # reset index
      numberBG = len(subset_ID["Breath.Group"].unique())
      grouped_vowels = subset_ID.groupby("Breath.Group")
      grouped_lists = grouped_vowels["Duration"].apply(list)
      grouped_lists = grouped_lists.reset_index()
      Vavg = np.zeros(len(grouped_lists))
      for i in range(0,len(grouped_lists)):
        Vavg[i] = np.mean(grouped_lists["Duration"][i])
      VarcoV = deltaV["DeltaV"][n]/Vavg
      VarcoV_avg = VarcoV.mean()
      participant.append([VarcoV_avg])
    participant = pd.DataFrame (participant, columns = ['VarcoV'])
    return(participant)

# **I. PWS vs. Control Participants**


**1. Calculate Rhythm Metrics**

In [ ]:
reading_vowels_fluent_pws, reading_vowels_fluent_control = split_groups(reading_vowels_fluent)
reading_consonants_fluent_pws, reading_consonants_fluent_control = split_groups(reading_consonants_fluent)
frog_vowels_fluent_pws, frog_vowels_fluent_control = split_groups(frog_vowels_fluent)
frog_consonants_fluent_pws, frog_consonants_fluent_control = split_groups(frog_consonants_fluent)

In [ ]:
nPVI_reading_fluent_pws = computenPVI(reading_vowels_fluent_pws)
nPVI_reading_fluent_control = computenPVI(reading_vowels_fluent_control)
nPVI_frog_fluent_pws = computenPVI(frog_vowels_fluent_pws)
nPVI_frog_fluent_control = computenPVI(frog_vowels_fluent_control)

In [ ]:
rPVI_reading_fluent_pws = computerPVI(reading_consonants_fluent_pws)
rPVI_reading_fluent_control = computerPVI(reading_consonants_fluent_control)
rPVI_frog_fluent_pws = computerPVI(frog_consonants_fluent_pws)
rPVI_frog_fluent_control = computerPVI(frog_consonants_fluent_control)

In [ ]:
percV_reading_fluent_pws = computePercV(reading_vowels_fluent_pws, reading_consonants_fluent_pws)
percV_frog_fluent_pws = computePercV(frog_vowels_fluent_pws, frog_consonants_fluent_pws)
percV_reading_fluent_control = computePercV(reading_vowels_fluent_control, reading_consonants_fluent_control)
percV_frog_fluent_control = computePercV(frog_vowels_fluent_control, frog_consonants_fluent_control)

In [ ]:
deltaC_reading_fluent_pws = computeDeltaC(reading_consonants_fluent_pws)
deltaC_frog_fluent_pws = computeDeltaC(frog_consonants_fluent_pws)
deltaC_reading_fluent_control = computeDeltaC(reading_consonants_fluent_control)
deltaC_frog_fluent_control = computeDeltaC(frog_consonants_fluent_control)

In [ ]:
deltaV_reading_fluent_pws = computeDeltaV(reading_vowels_fluent_pws)
deltaV_frog_fluent_pws = computeDeltaV(frog_vowels_fluent_pws)
deltaV_reading_fluent_control = computeDeltaV(reading_vowels_fluent_control)
deltaV_frog_fluent_control = computeDeltaV(frog_vowels_fluent_control)

In [ ]:
varcoC_reading_fluent_pws = computeVarcoC(reading_consonants_fluent_pws, deltaC_reading_fluent_pws)
varcoC_frog_fluent_pws = computeVarcoC(frog_consonants_fluent_pws, deltaC_frog_fluent_pws)
varcoC_reading_fluent_control = computeVarcoC(reading_consonants_fluent_control, deltaC_reading_fluent_control)
varcoC_frog_fluent_control = computeVarcoC(frog_consonants_fluent_control, deltaC_frog_fluent_control)

In [ ]:
varcoV_reading_fluent_pws = computeVarcoV(reading_vowels_fluent_pws, deltaV_reading_fluent_pws)
varcoV_frog_fluent_pws = computeVarcoV(frog_vowels_fluent_pws, deltaV_frog_fluent_pws)
varcoV_reading_fluent_control = computeVarcoV(reading_vowels_fluent_control, deltaV_reading_fluent_control)
varcoV_frog_fluent_control = computeVarcoV(frog_vowels_fluent_control, deltaV_frog_fluent_control)

**2. Assign Results to DataFrame**

In [ ]:
# create result dataframes
result_reading_fluent_pws = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_frog_fluent_pws = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_reading_fluent_control = pd.DataFrame(columns = ["ID","nPVI","rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_frog_fluent_control = pd.DataFrame(columns = ["ID","nPVI","rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])

In [ ]:
result_reading_fluent_pws["ID"] = pws_IDs_reading
result_reading_fluent_pws["nPVI"] = nPVI_reading_fluent_pws
result_reading_fluent_pws["rPVI"] = rPVI_reading_fluent_pws
result_reading_fluent_pws["%V"] = percV_reading_fluent_pws*100
result_reading_fluent_pws["DeltaC"] = deltaC_reading_fluent_pws
result_reading_fluent_pws["DeltaV"] = deltaV_reading_fluent_pws
result_reading_fluent_pws["VarcoC"] = varcoC_reading_fluent_pws*100
result_reading_fluent_pws["VarcoV"] = varcoV_reading_fluent_pws*100

In [ ]:
result_reading_fluent_control["ID"] = control_IDs_reading
result_reading_fluent_control["nPVI"] = nPVI_reading_fluent_control
result_reading_fluent_control["rPVI"] = rPVI_reading_fluent_control
result_reading_fluent_control["%V"] = percV_reading_fluent_control*100
result_reading_fluent_control["DeltaC"] = deltaC_reading_fluent_control
result_reading_fluent_control["DeltaV"] = deltaV_reading_fluent_control
result_reading_fluent_control["VarcoC"] = varcoC_reading_fluent_control*100
result_reading_fluent_control["VarcoV"] = varcoV_reading_fluent_control*100

In [ ]:
result_frog_fluent_pws["ID"] = pws_IDs_frog
result_frog_fluent_pws["nPVI"] = nPVI_frog_fluent_pws
result_frog_fluent_pws["rPVI"] = rPVI_frog_fluent_pws
result_frog_fluent_pws["%V"] = percV_frog_fluent_pws*100
result_frog_fluent_pws["DeltaC"] = deltaC_frog_fluent_pws
result_frog_fluent_pws["DeltaV"] = deltaV_frog_fluent_pws
result_frog_fluent_pws["VarcoC"] = varcoC_frog_fluent_pws*100
result_frog_fluent_pws["VarcoV"] = varcoV_frog_fluent_pws*100

In [ ]:
result_frog_fluent_control["ID"] = control_IDs_frog
result_frog_fluent_control["nPVI"] = nPVI_frog_fluent_control
result_frog_fluent_control["rPVI"] = rPVI_frog_fluent_control
result_frog_fluent_control["%V"] = percV_frog_fluent_control*100
result_frog_fluent_control["DeltaC"] = deltaC_frog_fluent_control
result_frog_fluent_control["DeltaV"] = deltaV_frog_fluent_control
result_frog_fluent_control["VarcoC"] = varcoC_frog_fluent_control*100
result_frog_fluent_control["VarcoV"] = varcoV_frog_fluent_control*100

**3. Plot Rhythm Metrics for Frog**

In [ ]:
result_frog_fluent_pws["Group"] = "PWS"
result_frog_fluent_control["Group"] = "Control"

In [ ]:
plot_fluent_frog = result_frog_fluent_pws.append(result_frog_fluent_control)
plot_fluent_frog["Condition"] = "frog"

In [ ]:
p_fluent = plot_fluent_frog.append(plot_fluent_frog)

In [ ]:
metrics = ['nPVI','rPVI','%V','DeltaC','DeltaV','VarcoC','VarcoV']

In [ ]:
for metric in metrics:
  sns.set_style("white")
  g = sns.catplot(x = "Condition", y= metric,
                  hue="Group",
                  data=p_fluent, kind="box",
                  height=4, aspect=.7);
  g.fig.subplots_adjust(top=0.8)
  g.fig.suptitle("Rhythmicity of Fluent Speech", fontsize=16, weight = "bold");

  new_labels = ['CWS', 'CWNS']
  for t, l in zip(g._legend.texts, new_labels):
      t.set_text(l)

**4. Plot Rhythm Metrics for Reading and Frog**

In [ ]:
result_reading_fluent_pws["Group"] = "PWS"
result_reading_fluent_control["Group"] = "Control"
result_frog_fluent_pws["Group"] = "PWS"
result_frog_fluent_control["Group"] = "Control"

In [ ]:
plot_fluent_reading = result_reading_fluent_pws.append(result_reading_fluent_control)
plot_fluent_reading["Condition"] = "reading"

In [ ]:
plot_fluent_frog = result_frog_fluent_pws.append(result_frog_fluent_control)
plot_fluent_frog["Condition"] = "frog"

In [ ]:
p_fluent = plot_fluent_reading.append(plot_fluent_frog)

In [ ]:
my_colors = ["#FF8000", "#009999"]
sns.set_palette(my_colors)

In [ ]:
metrics = ['nPVI','rPVI','%V','DeltaC','DeltaV','VarcoC','VarcoV']

In [ ]:
for metric in metrics:
  sns.set_style("white")
  g = sns.catplot(x = "Condition", y= metric,
                  hue="Group",
                  data=p_fluent, kind="box",
                  height=4, aspect=.7);
  g.fig.subplots_adjust(top=0.8)
  g.fig.suptitle("Rhythmicity of Fluent Speech", fontsize=16, weight = "bold");

  new_labels = ['CWS', 'CWNS']
  for t, l in zip(g._legend.texts, new_labels):
      t.set_text(l)

**5. Save Dataframe**

In [ ]:
# PWS vs. Control (fluent utternaces only)
result_reading_fluent_pws.to_excel("result_reading_fluent_pws.xlsx")
result_reading_fluent_control.to_excel("result_reading_fluent_control.xlsx")
result_frog_fluent_pws.to_excel("result_frog_fluent_pws.xlsx")
result_frog_fluent_control.to_excel("result_frog_fluent_control.xlsx")

# **II. Fluent vs. Disfluent Utterances (PWS)**

**0. Import**

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/3.MLU_Matched/

/content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/3.MLU_Matched


In [ ]:
# Fluent vs. Disfluent Utterances (PWS)
reading_vowels_pws_fluent = pd.read_excel("fluencyMatched_vowels_reading_pws.xlsx")
reading_consonants_pws_fluent = pd.read_excel("fluencyMatched_consonants_reading_pws.xlsx")
frog_vowels_pws_fluent = pd.read_excel("fluencyMatched_vowels_frog_pws.xlsx")
frog_consonants_pws_fluent = pd.read_excel("fluencyMatched_consonants_frog_pws.xlsx")

In [ ]:
#set up result path to save along the way
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics

dir = "4.MetricsResults"
if os.path.exists(dir) == False:
  os.mkdir(dir)

%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/4.MetricsResults/

**1. Calculate Rhythm Metrics**

In [ ]:
reading_vowels_fluent, reading_vowels_disfluent = split_fluency(reading_vowels_pws_fluent)
reading_consonants_fluent, reading_consonants_disfluent = split_fluency(reading_consonants_pws_fluent)
frog_vowels_fluent, frog_vowels_disfluent = split_fluency(frog_vowels_pws_fluent)
frog_consonants_fluent, frog_consonants_disfluent = split_fluency(frog_consonants_pws_fluent)

In [ ]:
nPVI_reading_fluent = computenPVI(reading_vowels_fluent)
nPVI_frog_fluent = computenPVI(frog_vowels_fluent)
nPVI_reading_disfluent = computenPVI(reading_vowels_disfluent)
nPVI_frog_disfluent = computenPVI(frog_vowels_disfluent)

In [ ]:
rPVI_reading_fluent = computerPVI(reading_consonants_fluent)
rPVI_frog_fluent = computerPVI(frog_consonants_fluent)
rPVI_reading_disfluent = computerPVI(reading_consonants_disfluent)
rPVI_frog_disfluent = computerPVI(frog_consonants_disfluent)

In [ ]:
percV_reading_fluent = computePercV(reading_vowels_fluent, reading_consonants_fluent)
percV_frog_fluent = computePercV(frog_vowels_fluent, frog_consonants_fluent)
percV_reading_disfluent = computePercV(reading_vowels_disfluent, reading_consonants_disfluent)
percV_frog_disfluent = computePercV(frog_vowels_disfluent, frog_consonants_disfluent)

In [ ]:
deltaC_reading_fluent = computeDeltaC(reading_consonants_fluent)
deltaC_frog_fluent = computeDeltaC(frog_consonants_fluent)
deltaC_reading_disfluent = computeDeltaC(reading_consonants_disfluent)
deltaC_frog_disfluent = computeDeltaC(frog_consonants_disfluent)

In [ ]:
deltaV_reading_fluent = computeDeltaV(reading_vowels_fluent)
deltaV_frog_fluent = computeDeltaV(frog_vowels_fluent)
deltaV_reading_disfluent = computeDeltaV(reading_vowels_disfluent)
deltaV_frog_disfluent = computeDeltaV(frog_vowels_disfluent)

In [ ]:
varcoC_reading_fluent = computeVarcoC(reading_consonants_fluent, deltaC_reading_fluent)
varcoC_frog_fluent = computeVarcoC(frog_consonants_fluent, deltaC_frog_fluent)
varcoC_reading_disfluent = computeVarcoC(reading_consonants_disfluent, deltaC_reading_disfluent)
varcoC_frog_disfluent = computeVarcoC(frog_consonants_disfluent, deltaC_frog_disfluent)

In [ ]:
varcoV_reading_fluent = computeVarcoV(reading_vowels_fluent, deltaV_reading_fluent)
varcoV_frog_fluent = computeVarcoV(frog_vowels_fluent, deltaV_frog_fluent)
varcoV_reading_disfluent = computeVarcoV(reading_vowels_disfluent, deltaV_reading_disfluent)
varcoV_frog_disfluent = computeVarcoV(frog_vowels_disfluent, deltaV_frog_disfluent)

**2. Assign Results to Rhythm Metrics**

In [ ]:
# create result dataframes
result_reading_fluent = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_frog_fluent = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_reading_disfluent = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_frog_disfluent = pd.DataFrame(columns = ["ID","nPVI", "rPVI","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])

In [ ]:
result_reading_fluent["ID"] = pws_IDs
result_reading_fluent["nPVI"] = nPVI_reading_fluent
result_reading_fluent["rPVI"] = rPVI_reading_fluent
result_reading_fluent["%V"] = percV_reading_fluent*100
result_reading_fluent["DeltaC"] = deltaC_reading_fluent
result_reading_fluent["DeltaV"] = deltaV_reading_fluent
result_reading_fluent["VarcoC"] = varcoC_reading_fluent*100
result_reading_fluent["VarcoV"] = varcoV_reading_fluent*100

In [ ]:
result_reading_disfluent["ID"] = pws_IDs
result_reading_disfluent["nPVI"] = nPVI_reading_disfluent
result_reading_disfluent["rPVI"] = rPVI_reading_disfluent
result_reading_disfluent["%V"] = percV_reading_disfluent*100
result_reading_disfluent["DeltaC"] = deltaC_reading_disfluent
result_reading_disfluent["DeltaV"] = deltaV_reading_disfluent
result_reading_disfluent["VarcoC"] = varcoC_reading_disfluent*100
result_reading_disfluent["VarcoV"] = varcoV_reading_disfluent*100

In [ ]:
result_frog_fluent["ID"] = pws_IDs
result_frog_fluent["nPVI"] = nPVI_frog_fluent
result_frog_fluent["rPVI"] = rPVI_frog_fluent
result_frog_fluent["%V"] = percV_frog_fluent*100
result_frog_fluent["DeltaC"] = deltaC_frog_fluent
result_frog_fluent["DeltaV"] = deltaV_frog_fluent
result_frog_fluent["VarcoC"] = varcoC_frog_fluent*100
result_frog_fluent["VarcoV"] = varcoV_frog_fluent*100

In [ ]:
result_frog_disfluent["ID"] = pws_IDs
result_frog_disfluent["nPVI"] = nPVI_frog_disfluent
result_frog_disfluent["rPVI"] = rPVI_frog_disfluent
result_frog_disfluent["%V"] = percV_frog_disfluent*100
result_frog_disfluent["DeltaC"] = deltaC_frog_disfluent
result_frog_disfluent["DeltaV"] = deltaV_frog_disfluent
result_frog_disfluent["VarcoC"] = varcoC_frog_disfluent*100
result_frog_disfluent["VarcoV"] = varcoV_frog_disfluent*100

**3. Plot Rhythm Metrics for Frog Only**

In [ ]:
result_reading_fluent["FluencyStatus"] = "fluent"
result_reading_disfluent["FluencyStatus"] = "disfluent"
result_frog_fluent["FluencyStatus"] = "fluent"
result_frog_disfluent["FluencyStatus"] = "disfluent"

In [ ]:
plot_fluent_frog = result_frog_disfluent.append(result_frog_fluent)
plot_fluent_frog["Condition"] = "frog"

In [ ]:
p_pws = plot_fluent_frog

In [ ]:
my_colors = ["#FF8000", "#FFCC99"]
sns.set_palette(my_colors)

In [ ]:
metrics = ['nPVI','rPVI','%V','DeltaC','DeltaV','VarcoC','VarcoV']

In [ ]:
for metric in metrics:
  sns.set_style("white")
  g = sns.catplot(x = "Condition", y=metric,
                  hue="FluencyStatus",
                  data=p_pws, kind="box",
                  height=4, aspect=.7);
  g.fig.subplots_adjust(top=0.8)
  g.fig.suptitle("Rhythmicity of Fluent vs. Disfluent Utterances in AWS", fontsize=16, weight = "bold");

  new_labels = ['Leading up to disfluency', 'Not leading up to disfluency']
  for t, l in zip(g._legend.texts, new_labels):
      t.set_text(l)

**4. Plot Rhythm Metrics for Reading and Frog**

In [ ]:
result_reading_fluent["FluencyStatus"] = "fluent"
result_reading_disfluent["FluencyStatus"] = "disfluent"
result_frog_fluent["FluencyStatus"] = "fluent"
result_frog_disfluent["FluencyStatus"] = "disfluent"

In [ ]:
plot_fluent_reading = result_reading_disfluent.append(result_reading_fluent)
plot_fluent_reading["Condition"] = "reading"

In [ ]:
plot_fluent_frog = result_frog_disfluent.append(result_frog_fluent)
plot_fluent_frog["Condition"] = "frog"

In [ ]:
p_pws = plot_fluent_reading.append(plot_fluent_frog)

In [ ]:
my_colors = ["#FF8000", "#FFCC99"]
sns.set_palette(my_colors)

In [ ]:
metrics = ['nPVI','rPVI','%V','DeltaC','DeltaV','VarcoC','VarcoV']

In [ ]:
for metric in metrics:
  sns.set_style("white")
  g = sns.catplot(x = "Condition", y=metric,
                  hue="FluencyStatus",
                  data=p_pws, kind="box",
                  height=4, aspect=.7);
  g.fig.subplots_adjust(top=0.8)
  g.fig.suptitle("Rhythmicity of Fluent vs. Disfluent Utterances in AWS", fontsize=16, weight = "bold");

  new_labels = ['Leading up to disfluency', 'Not leading up to disfluency']
  for t, l in zip(g._legend.texts, new_labels):
      t.set_text(l)

**5. Save Dataframe**

In [ ]:
# Fluent vs. Disfluent (PWS)
result_reading_fluent.to_excel("result_reading_fluent.xlsx")
result_reading_disfluent.to_excel("result_reading_disfluent.xlsx")
result_frog_fluent.to_excel("result_frog_fluent.xlsx")
result_frog_disfluent.to_excel("result_frog_disfluent.xlsx")